In [4]:
import numpy as np 
import pandas as pd
import os

train=pd.read_csv('Dataset/train.csv')
test=pd.read_csv('Dataset/test.csv')

In [5]:
train.columns

Index(['id', 'Marital status', 'Application mode', 'Application order',
       'Course', 'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['Target'] = encoder.fit_transform(train['Target'])

In [7]:
y_train=train['Target']
X_train=train.drop(['id','Target'],axis=1)
xid=test['id']
X_test=test.drop('id',axis=1)

In [8]:
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
models = [
    ('lgbm1', LGBMClassifier(learning_rate=0.1, n_estimators=700, num_leaves=31, colsample_bytree=0.8, subsample=0.8, reg_alpha=0.5, reg_lambda=0.1)),
    ('lgbm2', LGBMClassifier(learning_rate=0.05, n_estimators=750, num_leaves=41, colsample_bytree=0.7, subsample=0.7, reg_alpha=0.3, reg_lambda=0.2)),
    ('lgbm3', LGBMClassifier(learning_rate=0.2, n_estimators=650, num_leaves=21, colsample_bytree=0.9, subsample=0.9, reg_alpha=0.4, reg_lambda=0.3)),
    ('lgbm4', LGBMClassifier(learning_rate=0.15, n_estimators=550, num_leaves=61, colsample_bytree=0.85, subsample=0.75, reg_alpha=0.6, reg_lambda=0.1)),
    ('lgbm5', LGBMClassifier(learning_rate=0.1, n_estimators=600, num_leaves=51, colsample_bytree=0.75, subsample=0.85, reg_alpha=0.2, reg_lambda=0.5))
]
final_estimator = LogisticRegression(multi_class='multinomial', solver='lbfgs')
stackmodel1 = StackingClassifier(estimators=models, final_estimator=final_estimator)
stackmodel1.fit(X_train, y_train)
y_pred = stackmodel1.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004729 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1387
[LightGBM] [Info] Number of data points in the train set: 76518, number of used features: 36
[LightGBM] [Info] Start training from score -1.106880
[LightGBM] [Info] Start training from score -1.633484
[LightGBM] [Info] Start training from score -0.746204
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1387
[LightGBM] [Info] Number of data points in the train set: 76518, 

c:\Users\Dell\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print(y_pred)
y_pred = encoder.inverse_transform(y_pred)
print(y_pred)

[0 2 2 ... 0 0 0]
['Dropout' 'Graduate' 'Graduate' ... 'Dropout' 'Dropout' 'Dropout']


In [10]:
predictions = pd.DataFrame({'id': xid, 'Target': y_pred})
predictions.to_csv('submission.csv', index=False)